# Import required libraries

In [23]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

# Reading Data with sqlite

In [2]:
con = sqlite3.connect('E:\\IIT Kanpur\\Placements\\Projects\\Amazon Fine Food Reviews\\Dataset\\database.sqlite') 

filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3""", con) 

In [3]:
# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 0
    return 1

# Binary Classfication

In [4]:
#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


# Data deduplication

In [5]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
print(final.shape)

#Checking to see how much % of data still remains
print((final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100)

(364173, 10)
69.25890143662969


In [6]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(364171, 10)


1    307061
0     57110
Name: Score, dtype: int64

In [7]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,1,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,1,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,1,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,1,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,1,1018396800,A great way to learn the months,This is a book of poetry about the months of t...


# Loading W2V_TF-IDF matrix previously saved

In [8]:
w2v_tfidf = pd.read_csv("E:\\IIT Kanpur\\Placements\\Projects\\Amazon Fine Food Reviews\\w2v_tfidf.csv")

In [9]:
w2v_tfidf.drop(w2v_tfidf.columns[[0]], axis = 1, inplace = True)

In [10]:
a = np.asarray(final['Score'])
print(np.count_nonzero(np.isnan(a)))

w2v_tfidf['Score'] = a

0


In [11]:
a = np.asarray(final['Time'])
print(np.count_nonzero(np.isnan(a)))

w2v_tfidf['TimeStamp'] = a

0


# Random Smapling 5% from the whole data

In [12]:
w2v_tfidf.shape

(364171, 52)

In [13]:
np.random.seed(1)
rand_ind = np.random.choice(w2v_tfidf.shape[0], int(0.05 * w2v_tfidf.shape[0]), replace=False)

w2v_tfidf_rand = np.asarray(w2v_tfidf)[rand_ind, :]

In [14]:
X = w2v_tfidf_rand[:, :50]
Y = w2v_tfidf_rand[:, 50:51]

# Application of Model

## Logistic Regression

### Train/Test split in ratio 80 : 20

In [15]:
from sklearn.model_selection import train_test_split

X = w2v_tfidf_rand[:, :50]
Y = w2v_tfidf_rand[:, 50:51]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(14566, 50) (3642, 50) (14566, 1) (3642, 1)


In [36]:
X_train_df = pd.DataFrame(X_train)
Y_train_df = pd.DataFrame(Y_train)

In [41]:
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

logit_model = sm.Logit(Y_train_df, X_train_df)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.273537
         Iterations 8
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.365    
Dependent Variable: 0.0000           AIC:              8068.6781
Date:               2020-07-10 14:05 BIC:              8448.0003
No. Observations:   14566            Log-Likelihood:   -3984.3  
Df Model:           49               LL-Null:          -6277.5  
Df Residuals:       14516            LLR p-value:      0.0000   
Converged:          1.0000           Scale:            1.0000   
No. Iterations:     8.0000                                      
------------------------------------------------------------------
         Coef.    Std.Err.      z       P>|z|     [0.025    0.975]
------------------------------------------------------------------
0        1.1086     0.0771    14.3756   0.0000    0.9575    1.2598
1       -0.5107     0.0711    -7.1869   0.0000   -0.6500   -0.3714
2  

In [42]:
a = [9, 11,12,38,45,47]
X_train_df.drop(X_train_df.columns[a], axis = 1, inplace = True)
X_train_df.shape

(14566, 44)

In [43]:
logit_model = sm.Logit(Y_train_df, X_train_df)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.273846
         Iterations 8
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.365    
Dependent Variable: 0.0000           AIC:              8065.6719
Date:               2020-07-10 14:07 BIC:              8399.4755
No. Observations:   14566            Log-Likelihood:   -3988.8  
Df Model:           43               LL-Null:          -6277.5  
Df Residuals:       14522            LLR p-value:      0.0000   
Converged:          1.0000           Scale:            1.0000   
No. Iterations:     8.0000                                      
------------------------------------------------------------------
         Coef.    Std.Err.      z       P>|z|     [0.025    0.975]
------------------------------------------------------------------
0        1.0314     0.0713    14.4678   0.0000    0.8917    1.1711
1       -0.4864     0.0698    -6.9670   0.0000   -0.6233   -0.3496
2  

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logreg = LogisticRegression()
logreg.fit(X_train_df, Y_train_df)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
X_test_df = pd.DataFrame(X_test)
final_cols = X_train_df.columns
final_cols = list(final_cols)
X_test_df = X_test_df.iloc[:, final_cols]
X_test_df.shape

(3642, 44)

In [54]:
y_pred = logreg.predict(X_test_df)

In [55]:
y_pred.shape, Y_test.shape

((3642,), (3642, 1))

In [57]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

conf_mat = confusion_matrix(Y_test, y_pred)
print(conf_mat)

precision = conf_mat[1,1] / (conf_mat[1,1] + conf_mat[1,0])
recall = conf_mat[1,1] / (conf_mat[1,1] + conf_mat[0,1])
print(precision, recall)

[[ 261  329]
 [ 108 2944]]
0.9646133682830931 0.8994805988389857


In [58]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.71      0.44      0.54       590
         1.0       0.90      0.96      0.93      3052

    accuracy                           0.88      3642
   macro avg       0.80      0.70      0.74      3642
weighted avg       0.87      0.88      0.87      3642



## Random Forest

https://github.com/WillKoehrsen/Machine-Learning-Projects/blob/master/Random%20Forest%20Tutorial.ipynb

### Train/Test split in ratio 80 : 20

In [59]:
from sklearn.model_selection import train_test_split

X = w2v_tfidf_rand[:, :50]
Y = w2v_tfidf_rand[:, 50:51]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(14566, 50) (3642, 50) (14566, 1) (3642, 1)


In [60]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators =100, 
                               bootstrap = True,
                               max_features = 'sqrt')

model.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [61]:
y_pred = model.predict(X_test)

In [63]:
y_probs = model.predict_proba(X_test)[:, 1]

In [66]:
from sklearn.metrics import roc_auc_score

# Calculate roc auc
roc_value = roc_auc_score(Y_test, y_probs)

### Optimizing the parameters, best fit randomforest

In [69]:
from sklearn.model_selection import RandomizedSearchCV

# Hyperparameter grid
param_grid = {
    'n_estimators': np.linspace(10, 200).astype(int),
    'max_depth': [None] + list(np.linspace(3, 20).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [2, 5, 10],
    'bootstrap': [True, False]
}

# Estimator for use in random search
estimator = RandomForestClassifier(random_state = 0)

# Create the random search model
rs = RandomizedSearchCV(estimator, param_grid, n_jobs = -1, 
                        scoring = 'roc_auc', cv = 3, 
                        n_iter = 10, verbose = 1, random_state=0)

# Fit 
rs.fit(X_train, Y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  2.7min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [70]:
rs.best_params_

{'n_estimators': 192,
 'min_samples_split': 5,
 'max_leaf_nodes': 15,
 'max_features': 'auto',
 'max_depth': 11,
 'bootstrap': True}

In [71]:
best_model = rs.best_estimator_

In [73]:
y_pred = best_model.predict(X_test)
y_probs = best_model.predict_proba(X_test)[:, 1]

roc_value = roc_auc_score(Y_test, y_probs)

In [75]:
conf_mat = confusion_matrix(Y_test, y_pred)
print(conf_mat)

print('*'*100)

precision = conf_mat[1,1] / (conf_mat[1,1] + conf_mat[1,0])
recall = conf_mat[1,1] / (conf_mat[1,1] + conf_mat[0,1])
print(precision, recall)

print('*'*100)

print(classification_report(Y_test, y_pred))

[[  32  558]
 [   6 3046]]
****************************************************************************************************
0.9980340760157274 0.8451720310765816
****************************************************************************************************
              precision    recall  f1-score   support

         0.0       0.84      0.05      0.10       590
         1.0       0.85      1.00      0.92      3052

    accuracy                           0.85      3642
   macro avg       0.84      0.53      0.51      3642
weighted avg       0.84      0.85      0.78      3642



## Support Vector Machine

### Train/Test split in ratio 80 : 20

In [76]:
from sklearn.model_selection import train_test_split

X = w2v_tfidf_rand[:, :50]
Y = w2v_tfidf_rand[:, 50:51]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(14566, 50) (3642, 50) (14566, 1) (3642, 1)


In [80]:
from sklearn.svm import SVC # "Support Vector Classifier" 
clf = SVC(kernel='linear') 
  
# fitting x samples and y classes 
clf.fit(X_train, Y_train) 

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [81]:
y_pred = clf.predict(X_test)

In [84]:
conf_mat = confusion_matrix(Y_test, y_pred)
print(conf_mat)

print('*'*100)

precision = conf_mat[1,1] / (conf_mat[1,1] + conf_mat[1,0])
recall = conf_mat[1,1] / (conf_mat[1,1] + conf_mat[0,1])
print(precision, recall)

print('*'*100)

print(classification_report(Y_test, y_pred))

[[ 231  359]
 [  77 2975]]
****************************************************************************************************
0.9747706422018348 0.8923215356928614
****************************************************************************************************
              precision    recall  f1-score   support

         0.0       0.75      0.39      0.51       590
         1.0       0.89      0.97      0.93      3052

    accuracy                           0.88      3642
   macro avg       0.82      0.68      0.72      3642
weighted avg       0.87      0.88      0.86      3642



## Navie Bayes
https://towardsdatascience.com/implementing-naive-bayes-in-2-minutes-with-python-3ecd788803fe

### Train/Test split in ratio 80 : 20

In [85]:
from sklearn.model_selection import train_test_split

X = w2v_tfidf_rand[:, :50]
Y = w2v_tfidf_rand[:, 50:51]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(14566, 50) (3642, 50) (14566, 1) (3642, 1)


In [87]:
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

In [88]:
model = GaussianNB()
model.fit(X_train, Y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [89]:
y_pred = model.predict(X_test)

In [92]:
conf_mat = confusion_matrix(Y_test, y_pred)
print(conf_mat)

print('*'*100)

precision = conf_mat[1,1] / (conf_mat[1,1] + conf_mat[1,0])
recall = conf_mat[1,1] / (conf_mat[1,1] + conf_mat[0,1])
print(precision, recall)

print('*'*100)

print(classification_report(Y_test, y_pred))

[[ 332  258]
 [ 312 2740]]
****************************************************************************************************
0.8977719528178244 0.9139426284189459
****************************************************************************************************
              precision    recall  f1-score   support

         0.0       0.52      0.56      0.54       590
         1.0       0.91      0.90      0.91      3052

    accuracy                           0.84      3642
   macro avg       0.71      0.73      0.72      3642
weighted avg       0.85      0.84      0.85      3642



## Neural Network
https://medium.com/datadriveninvestor/building-neural-network-using-keras-for-classification-3a3656c726c1

### Train/Test split in ratio 80 : 20

In [93]:
from sklearn.model_selection import train_test_split

X = w2v_tfidf_rand[:, :50]
Y = w2v_tfidf_rand[:, 50:51]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(14566, 50) (3642, 50) (14566, 1) (3642, 1)


In [94]:
from keras import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [98]:
classifier = Sequential()

In [99]:
#First Hidden Layer
classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal', input_dim = 50))

#Second  Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

In [100]:
#Compiling the neural network
classifier.compile(optimizer ='adam', loss='binary_crossentropy', metrics =['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [101]:
#Fitting the data to the training dataset
classifier.fit(X_train, Y_train, batch_size=16, epochs=100)


Epoch 1/100
14566/14566 [==============================] - 2s 171us/step - loss: 0.3612 - accuracy: 0.8597
Epoch 2/100
14566/14566 [==============================] - 2s 127us/step - loss: 0.2790 - accuracy: 0.8884
Epoch 3/100
14566/14566 [==============================] - 2s 125us/step - loss: 0.2742 - accuracy: 0.8880
Epoch 4/100
14566/14566 [==============================] - 2s 123us/step - loss: 0.2715 - accuracy: 0.8908
Epoch 5/100
14566/14566 [==============================] - 2s 127us/step - loss: 0.2686 - accuracy: 0.8908
Epoch 6/100
14566/14566 [==============================] - 2s 126us/step - loss: 0.2660 - accuracy: 0.8917
Epoch 7/100
14566/14566 [==============================] - 2s 127us/step - loss: 0.2642 - accuracy: 0.8927
Epoch 8/100
14566/14566 [==============================] - 2s 125us/step - loss: 0.2625 - accuracy: 0.8928
Epoch 9/100
14566/14566 [==============================] - 2s 123us/step - loss: 0.2614 - accuracy: 0.8952
Epoch 10/100
14566/14566 [==========

In [102]:
eval_model=classifier.evaluate(X_train, Y_train)
eval_model

14566/14566 [==============================] - 0s 30us/step


[0.21226350829919813, 0.9146642684936523]

In [103]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred>0.5)

In [105]:
conf_mat = confusion_matrix(Y_test, y_pred)
print(conf_mat)

print('*'*100)

precision = conf_mat[1,1] / (conf_mat[1,1] + conf_mat[1,0])
recall = conf_mat[1,1] / (conf_mat[1,1] + conf_mat[0,1])
print(precision, recall)

print('*'*100)

print(classification_report(Y_test, y_pred))

[[ 299  291]
 [ 139 2913]]
****************************************************************************************************
0.9544560943643512 0.9091760299625468
****************************************************************************************************
              precision    recall  f1-score   support

         0.0       0.68      0.51      0.58       590
         1.0       0.91      0.95      0.93      3052

    accuracy                           0.88      3642
   macro avg       0.80      0.73      0.76      3642
weighted avg       0.87      0.88      0.87      3642



## K-nearest neighbors
https://www.datacamp.com/community/tutorials/k-nearest-neighbor-classification-scikit-learn

### Train/Test split in ratio 80 : 20

In [16]:
from sklearn.model_selection import train_test_split

X = w2v_tfidf_rand[:, :50]
Y = w2v_tfidf_rand[:, 50:51]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(14566, 50) (3642, 50) (14566, 1) (3642, 1)


In [17]:
from sklearn.neighbors import KNeighborsClassifier

In [46]:
# best is for k = 10 tried with other values but best is k = 10
model = KNeighborsClassifier(n_neighbors = 10)

model.fit(X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [47]:
y_pred = model.predict(X_test)

In [48]:
conf_mat = confusion_matrix(Y_test, y_pred)
print(conf_mat)

print('*'*100)

precision = conf_mat[1,1] / (conf_mat[1,1] + conf_mat[1,0])
recall = conf_mat[1,1] / (conf_mat[1,1] + conf_mat[0,1])
print(precision, recall)

print('*'*100)

print(classification_report(Y_test, y_pred))

[[ 181  409]
 [  84 2968]]
****************************************************************************************************
0.9724770642201835 0.8788865857269766
****************************************************************************************************
              precision    recall  f1-score   support

         0.0       0.68      0.31      0.42       590
         1.0       0.88      0.97      0.92      3052

    accuracy                           0.86      3642
   macro avg       0.78      0.64      0.67      3642
weighted avg       0.85      0.86      0.84      3642

